# Exercise 2.7 - Data Preparation for Dashboard Deployment

## Citi Bike NYC Expansion Dashboard - Final Dataset

**Author:** Saurabh Singh  
**Exercise:** Achievement 2, Exercise 2.7  
**Date:** February 2026

---

## Purpose

### Why create a reduced dataset?

**Deployment constraint:**
- GitHub has a 25 MB file size limit
- Original merged dataset exceeds this limit
- Need to create a smaller sample for deployment

**Strategy:**
- Random sampling to maintain data distribution
- Keep only columns needed for dashboard
- Set random seed for reproducibility
- Target: ~8% sample to stay under 25 MB

### What we'll do:

1. Load full dataset
2. Add season column for filtering
3. Drop unnecessary columns
4. Create random sample (seed=32)
5. Save reduced dataset for deployment

---

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np

---

## 2. Load Full Dataset

In [ ]:
# Load merged dataset
df = pd.read_csv('outputs/merged_citibike_weather_2022.csv')

In [ ]:
print(f"Original dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

---

## 3. Add Season Column

### Purpose:

Adding a season column enables filtering in the dashboard to analyze seasonal patterns.

### Season definitions:
- **Winter**: December, January, February, March, April (low demand months)
- **Spring**: Late April, May (transition)
- **Summer**: June, July, August, September (peak demand)
- **Fall**: October, November (transition)

In [ ]:
# Convert date to datetime and extract month
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['month'] = df['month'].astype('int')

In [ ]:
# Create season column
df['season'] = [
    "winter" if (month == 12 or 1 <= month <= 4)
    else "spring" if (4 < month <= 5)
    else "summer" if (6 <= month <= 9)
    else "fall"
    for month in df['month']
]

In [ ]:
# Verify season distribution
print("Season distribution:")
print(df['season'].value_counts())

---

## 4. Select Necessary Columns

### Columns to keep:

**For charts:**
- `date` - Time series analysis
- `start_station_name` - Station popularity
- `avgTemp` - Weather correlation

**For filtering:**
- `season` - Seasonal analysis
- `month` - Monthly patterns

**Derived columns:**
- `value` - Trip counting (will be added)

### Columns to drop:

All individual trip details not needed for aggregated analysis.

In [ ]:
# Load weather data
df_weather = pd.read_csv('outputs/weather_data_2022.csv')

# Convert dates
df_weather['date'] = pd.to_datetime(df_weather['date'])

# Merge weather data
df = df.merge(df_weather, on='date', how='left')

# Check columns
print("Columns after merge:", df.columns.tolist())
print("avgTemp present:", 'avgTemp' in df.columns)

In [ ]:
# Keep only necessary columns
columns_to_keep = ['date', 'start_station_name', 'avgTemp', 'season', 'month']
df_reduced = df[columns_to_keep].copy()

In [ ]:
print(f"Reduced columns: {df_reduced.columns.tolist()}")
print(f"Shape: {df_reduced.shape}")

---

## 5. Create Random Sample

### Sampling strategy:

**Reproducibility:**
- Set `np.random.seed(32)` for consistent results
- Same seed = same sample every time

**Sample size:**
- Target: ~8% of original data
- Method: Random selection using `np.random.rand()`

In [ ]:
# Set random seed for reproducibility
np.random.seed(32)

In [ ]:
# Create random sample (92% excluded, 8% kept)
exclude_mask = np.random.rand(len(df_reduced)) <= 0.92
df_sample = df_reduced[~exclude_mask]

In [ ]:
print(f"Original rows: {len(df_reduced):,}")
print(f"Sample rows: {len(df_sample):,}")
print(f"Percentage: {len(df_sample)/len(df_reduced)*100:.1f}%")

---

## 6. Add Value Column for Counting

Adding the `value` column enables trip counting in dashboard aggregations.

In [ ]:
# Add value column
df_sample['value'] = 1

In [ ]:
df_sample.head()

---

## 7. Create Daily Aggregation for Line Chart

Pre-aggregate daily data to improve dashboard performance.

In [ ]:
# Create daily aggregation
df_daily_full = df_sample.groupby('date', as_index=False).agg({
    'value': 'sum',
    'avgTemp': 'first'
})

df_daily_full.rename(columns={'value': 'bike_rides_daily'}, inplace=True)

In [ ]:
df_daily_full.head()

---

## 8. Save Reduced Dataset

In [ ]:
# Save reduced sample
df_sample.to_csv('outputs/reduced_data_to_plot.csv', index=False)
print(f"Saved: outputs/reduced_data_to_plot.csv")
print(f"Rows: {len(df_sample):,}")

In [ ]:
# Check file size
import os
file_size_mb = os.path.getsize('outputs/reduced_data_to_plot.csv') / (1024 * 1024)
print(f"File size: {file_size_mb:.2f} MB")
if file_size_mb < 25:
    print("✅ File is under 25 MB - ready for GitHub!")
else:
    print("⚠️ File exceeds 25 MB - need smaller sample")

---

## Summary

### Dataset prepared:

✅ **Reduced sample created** - Under 25 MB for GitHub deployment  
✅ **Season column added** - Enables seasonal filtering in dashboard  
✅ **Unnecessary columns removed** - Only dashboard essentials kept  
✅ **Random seed set** - Reproducible sampling (seed=32)  
✅ **Value column added** - Ready for trip counting  

### Files created:

- `outputs/reduced_data_to_plot.csv` - Main dashboard dataset

### Next steps:

1. Create multi-page Streamlit dashboard
2. Add seasonal filters and metrics
3. Include all visualizations with interpretations
4. Add recommendations page
5. Test locally
6. Deploy to Streamlit Community Cloud

The reduced dataset maintains the statistical properties of the full dataset while meeting deployment constraints!